# Open Street Map Transmission Lines

This notebook demonstrates downloading data from Open Street Map to fill gaps in the [Global Electricity Transmission And Distribution Lines](https://datacatalog.worldbank.org/dataset/derived-map-global-electricity-transmission-and-distribution-lines) (GETD) dataset.

To obtain GIS data from Open Street Map of any specified administrative area in the world, the [GeoFabrik](http://download.geofabrik.de/) download server is the easiest solution for querying data at the administrative level.

In this notebook, we address obtaining data for areas of eastern Russia and Greenland, two specific areas which have been subsequently updated in the Open Street Map data since the GETD was generated and published.

We will download the latest version of this data (currently 12/20/21) and extract the transmission line information.

### Mount Drive folder

Mount Drive folder for saving this data.

In [ ]:
!pip install google-cloud 
!pip install google-cloud-vision
!pip install google-colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Get packages

Install packages needed for analysis and import into workspace.

In [ ]:
!pip install esy-osmfilter
!pip install geopandas
import configparser, contextlib
import os, sys
import geopandas as gpd
from esy.osmfilter import osm_colors as CC
from esy.osmfilter import run_filter 
from esy.osmfilter import Node, Way, Relation
from esy.osmfilter import export_geojson

### Function to download power line data

Function to download data, filter to power lines and save as shapefile

In [ ]:
def get_power_lines(area_name):
  
 # Set input/output locations
  PBF_inputfile = os.path.join(os.getcwd(), '/content/'+area_name+'-latest.osm.pbf')
  JSON_outputfile = os.path.join(os.getcwd(),'/content/'+area_name+'-lines.json')
  
  # Pre-filter for transmission ("line"), distribution ("minor_line") and buried lines ("cable")
  prefilter   = {Node: {}, Way: {"power":["line", "minor_line", "cable"],}, Relation: {}}
  whitefilter = []
  blackfilter = []
  
  # Create initial data
  [Data,_]=run_filter('noname',
                    PBF_inputfile, 
                    JSON_outputfile, 
                    prefilter,
                    whitefilter,
                    blackfilter,
                    NewPreFilterData=True, 
                    CreateElements=False, 
                    LoadElements=False,
                    verbose=True)  
  
  # Check that data exists
  len(Data['Way'])
  
  # Get transmission, distribution and buried cable elements
  whitefilter = [(("power","line"),), (("power","minor_line"),), (("power","cable"),),]
  blackfilter = [((),),]
  
  # Apply filter
  [_,Elements]=run_filter('powerlines',
                        PBF_inputfile, 
                        JSON_outputfile, 
                        prefilter,
                        whitefilter, 
                        blackfilter, 
                        NewPreFilterData=False, 
                        CreateElements=True, 
                        LoadElements=False,
                        verbose=True)
  
  # Export data to geojson
  export_geojson(Elements['powerlines']['Way'],Data, filename='/content/'+area_name+'-lines.geojson',jsontype='Line')
  
  # Read data into geopandas
  gdf = gpd.read_file('/content/'+area_name+'-lines.geojson')
  
  # Write as shapefile
  gdf.to_file('/content/drive/MyDrive/ACEP_Arctic_Electrification/Data/Open_street_map_transmission/'+area_name+'_lines.shp')
  
  # Plot output
  gdf.plot()

### Function to download power plant data

Function to download data, filter to power plants, generators and storage and save as shapefile

In [ ]:
def get_power_plants(area_name):
  
 # Set input/output locations
  PBF_inputfile = os.path.join(os.getcwd(), '/content/'+area_name+'-latest.osm.pbf')
  JSON_outputfile = os.path.join(os.getcwd(),'/content/'+area_name+'-lines.json')
  
  # Pre-filter for power ("node"), plant ("plant"), generator (for wind) and storage
  prefilter   = {Node: {"power":["plant", "generator", "storage"],}, Way: {}, Relation: {}}
  whitefilter = []
  blackfilter = []
  
  # Create initial data
  [Data,_]=run_filter('noname',
                    PBF_inputfile, 
                    JSON_outputfile, 
                    prefilter,
                    whitefilter,
                    blackfilter,
                    NewPreFilterData=True, 
                    CreateElements=False, 
                    LoadElements=False,
                    verbose=True)  
  
  # Check that data exists
  len(Data['Node'])
  
  # Get plant, generator and storage elements
  whitefilter = [(("power","plant"),), (("power","generator"),), (("power","storage"),),]
  blackfilter = [((),),]
  
  # Apply filter
  [_,Elements]=run_filter('powerplants',
                        PBF_inputfile, 
                        JSON_outputfile, 
                        prefilter,
                        whitefilter, 
                        blackfilter, 
                        NewPreFilterData=False, 
                        CreateElements=True, 
                        LoadElements=False,
                        verbose=True)
  
  # Export data to geojson
  export_geojson(Elements['powerplants']['Node'],Data, filename='/content/'+area_name+'-powerplants.geojson',jsontype='Point')
  
  # Read data into geopandas
  gdf = gpd.read_file('/content/'+area_name+'-powerplants.geojson')
  
  # Write as shapefile
  gdf.to_file('/content/drive/MyDrive/ACEP_Arctic_Electrification/Data/Open_street_map_transmission/'+area_name+'_powerplants.shp')
  
  # Plot output
  gdf.plot()

## Get Alaska Data

In [ ]:
!wget http://download.geofabrik.de/north-america/us/alaska-latest.osm.pbf

In [ ]:
get_power_lines('alaska')

## Get Faroe Islands Data

In [ ]:
!wget http://download.geofabrik.de/europe/faroe-islands-latest.osm.pbf

In [ ]:
get_power_lines('faroe-islands')